In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.pyplot import plot, draw, show
from scipy.linalg import lu
import scipy.linalg as la
import random
import simpy


p $p$ $\rho$

In [2]:
def plotpretify(ax, maxx, minx=0):
    ax.spines["top"].set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["bottom"].set_visible(False)

    ax.tick_params(
        axis="both",
        which="both",
        bottom=False,
        top=False,
        labelbottom=True,
        left=False,
        right=False,
        labelleft=True)
    ax.grid(True)
    
    ax.set_xlim(minx, maxx)

In [115]:
RANDOM_SEED = 42
N_helpers = [1, 2, 4]  # Number of machines in the queue
waitTime = 20  # Minutes it takes to help a customer
lambdaIAT = 6  # Create a customer every ~7 minutes
SIM_TIME = 80  # Simulation time in minutes


class Queue(object):
    def __init__(self, env, N, waitTime):
        self.env = env
        self.machine = simpy.Resource(env, N)
        self.waitTime = waitTime
        self.customerHelped = 0
        self.helperN = N

    def helped(self, customer, customerWait=waitTime):
        #         print([dir(node) if node else "" for node in self.machine.queue])
        yield self.env.timeout(customerWait)


#         queuelist = [(user.name, user.time) for user in self.machine.queue]
#         print(queuelist)

#         print(f"{customer} helped at {self.env.now}")


def customer(env, name, cw, id, customerWait):
    customerWait = random.randint(waitTime - 5, waitTime + 5)
    #     print('%s arrives at the queue at %.2f.' % (name, env.now))
    enterQueue = env.now
    with cw.machine.request() as request:
        request.name = name
        request.time = customerWait
        yield request

        #         print('%s turn begins at %.2f.' % (name, env.now))
        customerStat[cw.helperN].append(env.now - enterQueue)
        yield env.process(cw.helped(name, customerWait=customerWait))


#         print('%s leaves the queue at %.2f.' % (name, env.now))


def setup(env, N, waitTime, lambdaIAT):
    queue = Queue(env, N, waitTime)

    # Create more customers while the simulation is running
    while queue.customerHelped < 1000:
        s = np.random.poisson(lambdaIAT, 1000)
        yield env.timeout(s[queue.customerHelped])
        queue.customerHelped += 1
        env.process(
            customer(
                env,
                'customer %d' % queue.customerHelped,
                queue,
                id=queue.customerHelped,
                customerWait = 1000 - queue.customerHelped))


random.seed(RANDOM_SEED)
customerStat = {}
for j in range(1):
    for N in N_helpers:
        #         print(f"******** N={N} Helpers ********")
        customerStat.setdefault(N, [])
        env = simpy.Environment()
        env.process(setup(env, N, waitTime, lambdaIAT))
        env.run()

[
    print(
        f"run{N} average wait: {np.mean(customerStat[N])}, variance: {np.var(customerStat[N])}"
    ) for N in customerStat
]

run1 average wait: 6952.216, variance: 16313705.427344
run2 average wait: 2003.802, variance: 1340179.116796
run4 average wait: 1.616, variance: 9.100544000000001


[None, None, None]

In [116]:
RANDOM_SEED = 42
N_helpers = [1, 2, 4]  # Number of machines in the queue
waitTime = 20  # Minutes it takes to help a customer
lambdaIAT = 6 # Create a customer every ~7 minutes
SIM_TIME = 80  # Simulation time in minutes


class Queue(object):
    def __init__(self, env, N, waitTime):
        self.env = env
        self.machine = simpy.PriorityResource(env, N)
        self.waitTime = waitTime
        self.customerHelped = 0
        self.helperN = N

    def helped(self, customer, customerWait=waitTime):
        #         print([dir(node) if node else "" for node in self.machine.queue])
        yield self.env.timeout(customerWait)
#         queuelist = [(user.name, user.time) for user in self.machine.queue]
#         print(queuelist)


#         print(f"{customer} helped at {self.env.now}")


def customer(env, name, cw, id, customerWait):
    customerWait = random.randint(waitTime - 5, waitTime + 5)
    #     print('%s arrives at the queue at %.2f.' % (name, env.now))
    enterQueue = env.now
    with cw.machine.request(priority=customerWait) as request:
        request.name = name
        request.time = customerWait
        yield request

        #         print('%s turn begins at %.2f.' % (name, env.now))
        customerStat[cw.helperN].append(env.now - enterQueue)
        #         yield env.process(cw.helped(name))
        yield env.process(cw.helped(name, customerWait=customerWait))


#         print('%s leaves the queue at %.2f.' % (name, env.now))


def setup(env, N, waitTime, lambdaIAT):
    queue = Queue(env, N, waitTime)

    # Create more customers while the simulation is running
    while queue.customerHelped < 1000:
        s = np.random.poisson(lambdaIAT, 1000)
        yield env.timeout(s[queue.customerHelped])
        queue.customerHelped += 1
        env.process(
            customer(
                env,
                'customer %d' % queue.customerHelped,
                queue,
                id=queue.customerHelped,
                customerWait=1000-queue.customerHelped))


random.seed(RANDOM_SEED)
customerStat = {}
for j in range(1):
    for N in N_helpers:
#         print(f"******** N={N} Helpers ********")
        customerStat.setdefault(N, [])
        env = simpy.Environment()
        env.process(setup(env, N, waitTime, lambdaIAT))
        env.run()

[
    print(f"run{N} average wait: {np.mean(customerStat[N])}, variance: {np.var(customerStat[N])}")
    for N in customerStat
]

run1 average wait: 6191.364, variance: 32021126.915503997
run2 average wait: 1667.435, variance: 6166069.243775
run4 average wait: 1.371, variance: 9.995359


[None, None, None]